<a href="https://colab.research.google.com/github/pansplawik/eegParkinsonCognitiveDisorders/blob/master/thirdVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne
!pip install autoreject

Importowanie pakietów

In [2]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import csv
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from mne.decoding import PSDEstimator
from autoreject import AutoReject
from mne.decoding import Vectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold, cross_val_predict
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# Models
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
ar = AutoReject()

Ściezki plików

In [ ]:
labels = pd.read_csv('/content/drive/MyDrive/dyplom/dataset.csv', sep=';')
labels['PD'] = labels['PD'].replace(2, 1)
for a in labels['ID']:
  print(a)
path='/content/drive/MyDrive/dyplom/EEG'

In [4]:
filenames = sorted(os.listdir(path))
print(filenames)

['PD129.edf', 'PD130.edf', 'PD131.edf', 'PD132.edf', 'PD133.edf', 'PD134.edf', 'PD135.edf', 'PD136.edf', 'PD137.edf', 'PD138.edf', 'PD139.edf', 'PD140.edf', 'PD141.edf', 'PD142.edf', 'PD143.edf', 'PD144.edf', 'PD145.edf', 'PD146.edf', 'PD147.edf', 'PD148.edf', 'PD149.edf', 'PD150.edf', 'PD151.edf', 'PD152.edf', 'PD153.edf', 'PD154.edf', 'PD155.edf', 'PD156.edf', 'PD157.edf', 'PD158.edf', 'PD159.edf', 'PD160.edf', 'PD161.edf', 'PD162.edf', 'PD163.edf', 'PD164.edf', 'PD165.edf', 'PD166.edf', 'PD167.edf', 'PD170.edf', 'PD171.edf', 'PD172.edf', 'PD173.edf', 'PD174.edf', 'PD175.edf', 'PD176.edf', 'PD177.edf', 'PD178.edf', 'PD180.edf', 'PD181.edf', 'PD182.edf', 'PD183.edf', 'PD184.edf', 'PD185.edf', 'PD186.edf', 'PD187.edf', 'PD188.edf', 'PD189.edf', 'PD190.edf', 'PD191.edf', 'PD192.edf', 'PD193.edf', 'PD194.edf', 'PD195.edf', 'PD196.edf', 'PD197.edf', 'PD198.edf', 'PD199.edf', 'PD200.edf', 'PD201.edf', 'PD202.edf', 'PD203.edf', 'PD204.edf', 'PD205.edf', 'PD206.edf', 'PD207.edf', 'PD208.edf'

Preprocesing danych

In [5]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    liczba=1
    filenames = sorted(os.listdir(path))
    for filename, label in zip(filenames, labels['PD']):
        print(filename)
        if filename.endswith('.edf'):
            filepath = os.path.join(path, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            raw.info.set_montage('standard_1020')
            raw.info['bads'].append('Oz')
            annotations = raw.annotations
            for i in range(len(annotations)):
              if annotations.description[i] == "break":
                annotations.description[i] = "BAD" 
              elif annotations.description[i] == 'Break end':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Miesnie':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Break start':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Ruch':
                annotations.description[i] = "BAD"
            raw.set_annotations(annotations)
            raw.filter(1, 40, fir_design='firwin')
            events = mne.make_fixed_length_events(raw, duration=1.0)
            reject = dict(     
              eeg=82e-6,       
              )
            epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1,preload=True,reject_by_annotation='BAD',reject=reject)
            epochs_array.append([epochs,label])      
            del epochs
            del raw
            del annotations
    return epochs_array


PSD

In [6]:
def psds(epochs_array, labels):
    data = []
    for epochs, label in zip(epochs_array, labels):
      if epochs is not None and len(epochs) > 0:
        epo_spectrum = epochs.compute_psd(method='multitaper', fmin=1, fmax=40, picks='eeg')
        psds, freqs = epo_spectrum.get_data(return_freqs=True)
        for p in psds:
          data.append([p, label])
        del psds
        del freqs
        del epo_spectrum
    return data

In [ ]:
data = load_epochs_from_dir('/content/drive/MyDrive/dyplom/EEG',labels)
df = pd.DataFrame(data, columns=['value', 'label'])
df.to_csv('data.csv', index=False)

In [8]:
psd = psds([epochs for epochs, label in data], [label for epochs, label in data])
df = pd.DataFrame(psd, columns=['psds', 'label'])
df.to_csv('psd.csv', index=False)

Tworzenie zestawów

In [9]:
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(df['psds'], df['label'], test_size=0.2, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# Tworzenie i trenowanie modelu SVM
clf_svm_pip = make_pipeline(Vectorizer(), StandardScaler(), SVC())
clf_svm_pip.fit(X_train_2d, y_train)

# Przewidywanie etykiet dla danych testowych
predictions_svm = clf_svm_pip.predict(X_test_2d)

# Raport klasyfikacji
report_svm = classification_report(y_test, predictions_svm, target_names=['0', 'dwsd'])
print('SVM Classification Report:\n{}'.format(report_svm))

# Dokładność modelu SVM
acc_svm = accuracy_score(y_test, predictions_svm)
print("Accuracy of SVM model: {}".format(acc_svm))

# Obliczanie precyzji, czułości i miary F1
precision_svm, recall_svm, fscore_svm, support_svm = precision_recall_fscore_support(y_test, predictions_svm, average='macro')
print('Precision: {0}, Recall: {1}, F1-score: {2}'.format(precision_svm, recall_svm, fscore_svm))


SVM Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.80      0.74      7583
        dwsd       0.77      0.64      0.70      7583

    accuracy                           0.72     15166
   macro avg       0.73      0.72      0.72     15166
weighted avg       0.73      0.72      0.72     15166

Accuracy of SVM model: 0.7228010022418568
Precision: 0.7288013043562509, Recall: 0.7228010022418567, F1-score: 0.7209716261525427


Tworzenie modelu

In [ ]:
svr = SVC(kernel='linear', C=1)
svr.fit(X_train_2d, y_train)
y_pred_test = svr.predict(X_test_2d)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", accuracy)

Accuracy: 0.5


Trening i wizualizacja wyników

In [12]:
from joblib import dump, load
dump(clf_svm_pip, 'oficjalnyPierwszy.joblib') 

['oficjalnyPierwszy.joblib']